<div style="text-align:center"> <img src="https://sun9-45.userapi.com/impg/Flbnug2OUli1ecXsoIKeUasIGXGj_5hqjX4cRg/z2nfz8-b3a0.jpg?size=2560x1153&quality=96&sign=0536543610f1655d967af88dbc775e98&type=album" width="800">

In [ ]:
import os
import sys 
from tqdm import tqdm
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image
import pydicom

import numpy as np
import nibabel as nib
import matplotlib.pyplot as plt
import SimpleITK as sitk

train_path = '../input/rsna-miccai-brain-tumor-radiogenomic-classification/train/'

In [ ]:
train_dirs = os.listdir(train_path)

In [ ]:
plt.figure(figsize=(14,7))
plt.subplot(121)
plt.imshow(pydicom.dcmread(f'{train_path + train_dirs[0]}/T2w/Image-111.dcm').pixel_array)
plt.subplot(122)
plt.imshow(pydicom.dcmread(f'{train_path + train_dirs[0]}/T1w/Image-111.dcm').pixel_array)

Different spatial orientation of images is inconvinient. But we can make it the same. 

# 1. (not really important part) Affine matrix and simple resampling

Each DICOM file stores information about its orientation in the scanner space (which is basically the real world, with the center of the coordinate system in the magnet isocenter). 

Let's convert one of the DICOM-series to a NIfTI file to see it most clearly. There are numerous ways to do it, we'll use the functionality of the SimpleITK library. 

In [ ]:
reader = sitk.ImageSeriesReader()
reader.LoadPrivateTagsOn()

In [ ]:
filenamesDICOM = reader.GetGDCMSeriesFileNames(f'{train_path}/{train_dirs[2]}/T1w')
reader.SetFileNames(filenamesDICOM)
t1_sitk = reader.Execute()

In [ ]:
sitk.WriteImage(t1_sitk,'t1.nii')

Now we can load it with Nibabel. There's a lot of stuff in the `nibabel.nifti1.Nifti1Image` object, but the two essential things are voxel array and affine matrix. 

In [ ]:
t1_nib = nib.load('t1.nii')
t1_nib

In [ ]:
t1_nib_array = t1_nib.get_fdata() #the voxel array
t1_nib_array[:3]

In [ ]:
t1_nib_array.shape

In [ ]:
plt.imshow(t1_nib_array[:,:,t1_nib_array.shape[2]//2])

So, this is the voxel array. Its orientation in scanner space is encoded in the affine matrix:

In [ ]:
np.set_printoptions(precision=4, suppress=True)
t1_nib.affine

The first 3х3 part of the matrix provides information about rotation and scaling. The fourth column tells us about translation.

So, if we want to know the coordinates of a voxel b=[2,5,10] in the scanner space, we can just calculate its dot product with upper left 3х3 corner of the affine matrix, and then sum the result with the translation vector 𝑡 (which is the forth column of the affine matrix).

$$ 1) \,\,\,\,\, 
    \begin{bmatrix} 
        { A }_{ 00 } & { A }_{ 01 } & { A }_{ 02 }\\ 
        { A }_{ 10 } & { A }_{ 11 } & { A }_{ 12 }\\ 
        { A }_{ 20 } & { A }_{ 21 } & { A }_{ 22 }\end{bmatrix} \cdot
   \begin{bmatrix} 
        { b_0 } \\ { b_1 }  \\ { b_2 } \end{bmatrix} =
   \begin{bmatrix}      
        { A }_{ 00 } \\ { A }_{ 10 } \\ { A }_{ 20 } \end{bmatrix} \cdot { b_0 } + 
   \begin{bmatrix}      
        { A }_{ 01 } \\ { A }_{ 11 } \\ { A }_{ 21 } \end{bmatrix} \cdot { b_1 } +    
   \begin{bmatrix}      
        { A }_{ 02 } \\ { A }_{ 12 } \\ { A }_{ 22 } \end{bmatrix} \cdot { b_2 } =    
   \begin{bmatrix}   
   { A }_{ 00 } { b_0 } + { A }_{ 01 } { b_1 } + { A }_{ 02 } { b_2 } \\
   { A }_{ 10 } { b_0 } + { A }_{ 11 } { b_1 } + { A }_{ 12 } { b_2 } \\
   { A }_{ 20 } { b_0 } + { A }_{ 21 } { b_1 } + { A }_{ 22 } { b_2 } \end{bmatrix} = 
   \begin{bmatrix} 
        { x_0 } \\ { x_1 }  \\ { x_2 } \end{bmatrix} $$ <br/>


$$ 2) \,\,\,\,\, 
    \begin{bmatrix} 
        { x_0 } \\ { x_1 }  \\ { x_2 } \end{bmatrix}  + 
    \begin{bmatrix} 
        { t_0 } \\ { t_1 }  \\ { t_2 } \end{bmatrix} = 
    \begin{bmatrix} 
        { x_0+t_0 } \\ { x_1+t_1 }  \\ { x_2+t_2 } \end{bmatrix} = 
    \begin{bmatrix} 
        { a } \\ { b }  \\ { c } \end{bmatrix}
        $$

In [ ]:
t1_nib.affine[:3,:3] @ np.array([2,5,10]) + t1_nib.affine[:3,3]

The last row in the affine matrix is always the [0,0,0,1] like in the identity matrix, it's just there to make the matrix square so we could use it as a linear operator. Therefore we can also just do this:

In [ ]:
t1_nib.affine @ np.array([2,5,10,1]) #(add 1 as a fourth coordinate)

Apparently, as all series in the study have their affines linked to the same scanner space, we can resample them all into the same voxel space. 

In [ ]:
filenamesDICOM = reader.GetGDCMSeriesFileNames(f'{train_path}/{train_dirs[2]}/FLAIR')
reader.SetFileNames(filenamesDICOM)
flair_sitk = reader.Execute()
sitk.WriteImage(flair_sitk,'flair.nii')

flair_nib = nib.load('flair.nii')
flair_nib_array = flair_nib.get_fdata()

In [ ]:
plt.figure(figsize=(12,6))
plt.subplot(121)
plt.imshow(t1_nib_array[:,:,t1_nib_array.shape[2]//2])
plt.subplot(122)
plt.imshow(flair_nib_array[:,:,flair_nib_array.shape[2]//2])

In [ ]:
from nilearn.image import resample_img

In [ ]:
%%time 
flair_resampled = resample_img(flair_nib, target_affine=t1_nib.affine, target_shape=t1_nib.shape)
flair_resampled_array = flair_resampled.get_fdata()

In [ ]:
plt.figure(figsize=(12,6))
plt.subplot(121)
plt.imshow(t1_nib_array[:,:,t1_nib_array.shape[2]//2])
plt.subplot(122)
plt.imshow(flair_resampled_array[:,:,flair_resampled_array.shape[2]//2])

It works rather slowly, but there is a faster way. 

# 2. SimpleITK resampling

In [ ]:
filenamesDICOM = reader.GetGDCMSeriesFileNames(f'{train_path}/{train_dirs[2]}/T1w')
reader.SetFileNames(filenamesDICOM)
t1_sitk = reader.Execute()
t1_sitk

So, the `SimpleITK.SimpleITK.Image` also contains information about voxel space orientation in the real world. It's not stored by the means of affine matrix though. Here, it goes in a few pieces:

In [ ]:
t1_sitk.GetOrigin() # which is a translation column from the affine matrix, but negative

In [ ]:
t1_sitk.GetSpacing() # which is how far away the voxel centers are from one another along each of the axes

In [ ]:
t1_sitk.GetDirection() # a flatten cosine matrix which shows rotation of voxel space axes relative to scanner space

We could actually extract all that information from the previously seen affine matrix. For example, knowing that each column affects one of the resulting voxel coordinates, we could get information about scaling (aka spacing, in this case).

In [ ]:
x_scale = np.linalg.norm(t1_nib.affine[:,0])
y_scale = np.linalg.norm(t1_nib.affine[:,1])
z_scale = np.linalg.norm(t1_nib.affine[:,2])
print(x_scale, y_scale, z_scale)

Also, if we divide each column by the corresponding spacing number, we'll get a cosine matrix.

In [ ]:
t1_pure_rotation = np.hstack((t1_nib.affine[:,0].reshape(-1,1)/x_scale,
                   t1_nib.affine[:,1].reshape(-1,1)/y_scale,
                   t1_nib.affine[:,2].reshape(-1,1)/z_scale,
                   t1_nib.affine[:,3].reshape(-1,1)))
t1_pure_rotation[:3,:3]

The first two rows differ in sign from the SimpleITK cosine matrix, I'm not sure why. It has something to do with a rotation direction. 

Btw, this information can be acquired from DICOM files. There's a tag for this:

In [ ]:
cosine_from_dcm = pydicom.dcmread(filenamesDICOM[1]).ImageOrientationPatient
cosine_from_dcm

As you can see, it has information only about two rows (6 numbers instead of 9). We can calculate the third row though. It's perpendicular two the first two row vectors, so we can calculate their cross product:

In [ ]:
np.cross(cosine_from_dcm[:3], cosine_from_dcm[3:])

Back to resampling. It's a bit more complicated but still pretty straightforward:

In [ ]:
def resample(image, ref_image):

    resampler = sitk.ResampleImageFilter()
    resampler.SetReferenceImage(ref_image)
    resampler.SetInterpolator(sitk.sitkLinear)
    
    resampler.SetTransform(sitk.AffineTransform(image.GetDimension()))

    resampler.SetOutputSpacing(ref_image.GetSpacing())

    resampler.SetSize(ref_image.GetSize())

    resampler.SetOutputDirection(ref_image.GetDirection())

    resampler.SetOutputOrigin(ref_image.GetOrigin())

    resampler.SetDefaultPixelValue(image.GetPixelIDValue())

    resamped_image = resampler.Execute(image)
    
    return resamped_image

In [ ]:
flair_resampled = resample(flair_sitk, t1_sitk)

In [ ]:
t1_sitk_array = sitk.GetArrayFromImage(t1_sitk)
flair_resampled_array = sitk.GetArrayFromImage(flair_resampled)

In [ ]:
plt.figure(figsize=(12,6))
plt.subplot(121)
plt.imshow(t1_sitk_array[t1_sitk_array.shape[0]//2,:,:])
plt.subplot(122)
plt.imshow(flair_resampled_array[flair_resampled_array.shape[0]//2,:,:])

It works **much** faster. 

In [ ]:
def normalize(data):
    return (data - np.min(data)) / (np.max(data) - np.min(data))

In [ ]:
%%time
filenamesDICOM = reader.GetGDCMSeriesFileNames(f'{train_path}/{train_dirs[2]}/T1w')
reader.SetFileNames(filenamesDICOM)
t1_sitk = reader.Execute()

filenamesDICOM = reader.GetGDCMSeriesFileNames(f'{train_path}/{train_dirs[2]}/FLAIR')
reader.SetFileNames(filenamesDICOM)
flair_sitk = reader.Execute()

filenamesDICOM = reader.GetGDCMSeriesFileNames(f'{train_path}/{train_dirs[2]}/T2w')
reader.SetFileNames(filenamesDICOM)
t2_sitk = reader.Execute()

flair_resampled = resample(flair_sitk, t1_sitk)
t2_resampled = resample(t2_sitk, t1_sitk)

t1_sitk_array = normalize(sitk.GetArrayFromImage(t1_sitk))
flair_resampled_array = normalize(sitk.GetArrayFromImage(flair_resampled))
t2_resampled_array = normalize(sitk.GetArrayFromImage(t2_resampled))

stacked = np.stack([t1_sitk_array, t2_resampled_array, flair_resampled_array,])

to_rgb = stacked[:,t1_sitk_array.shape[0]//2,:,:].transpose(1,2,0)
im = Image.fromarray((to_rgb * 255).astype(np.uint8))

In [ ]:
im

Let's resample some more volumes and look at some more pictures. 

In [ ]:
reader = sitk.ImageSeriesReader()
reader.LoadPrivateTagsOn()
filenamesDICOM = reader.GetGDCMSeriesFileNames(f'{train_path}/{train_dirs[20]}/T1w')
reader.SetFileNames(filenamesDICOM)
t1_reference = reader.Execute()

In [ ]:
sitk.GetArrayFromImage(t1_reference).shape

In [ ]:
plt.imshow(sitk.GetArrayFromImage(t1_reference)[15,:,:])

In [ ]:
fig, axs = plt.subplots(5,4, figsize=(12, 18), facecolor='w', edgecolor='k', dpi=100)
axs = axs.ravel()

for i, folder in enumerate(train_dirs[:20]):
    filenamesDICOM = reader.GetGDCMSeriesFileNames(f'{train_path}/{folder}/FLAIR')
    reader.SetFileNames(filenamesDICOM)
    flair = reader.Execute()
    
    flair_resampled = resample(flair, t1_reference)
    flair_resampled = normalize(sitk.GetArrayFromImage(flair_resampled))
        
    filenamesDICOM = reader.GetGDCMSeriesFileNames(f'{train_path}/{folder}/T1wCE')
    reader.SetFileNames(filenamesDICOM)
    t1 = reader.Execute()

    filenamesDICOM = reader.GetGDCMSeriesFileNames(f'{train_path}/{folder}/T2w')
    reader.SetFileNames(filenamesDICOM)
    t2 = reader.Execute()
        
    t1_resampled = resample(t1, t1_reference)
    t1_resampled = normalize(sitk.GetArrayFromImage(t1_resampled))

    t2_resampled = resample(t2, t1_reference)
    t2_resampled = normalize(sitk.GetArrayFromImage(t2_resampled))
        
    stacked = np.stack([t1_resampled, t2_resampled, flair_resampled])
         
    to_rgb = stacked[:,18,:,:].transpose(1,2,0)
    im = Image.fromarray((to_rgb * 255).astype(np.uint8))
    axs[i].imshow(im)

You can also resample these images into coronal plane or saggital plane. Or resample them into axial plane, but using another patient as reference (who knows, maybe it's a good way to augment the data). 